In [1]:
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPUfrom sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

23/02/23 22:32:22 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
23/02/23 22:32:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-75f9e559-3c1a-413d-87c9-625215bd1dfa;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#l

23/02/23 22:32:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/23 22:32:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv

In [3]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

In [4]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("text_train.csv")
trainDataset.show(10, truncate=50)

+--------------------------------+--------+
|                     description|category|
+--------------------------------+--------+
|           i want to make coffee|      DA|
|                      where am i|      LR|
|              what can i do here|      AR|
|                    prepare meal|      DA|
|                use refrigerator|      DA|
|                         use fan|      DA|
|                        use oven|      DA|
|                       use stove|      DA|
|     i would like to wash sheets|      DA|
|i would like to watch television|      DA|
+--------------------------------+--------+
only showing top 10 rows



In [5]:
testDataset = spark.read \
      .option("header", True) \
      .csv("text_test.csv")
testDataset.show(10, truncate=50)

+--------------------------------+--------+
|                     description|category|
+--------------------------------+--------+
|           i want to make coffee|      DA|
|                      where am i|      LR|
|              what can i do here|      AR|
|                    prepare meal|      DA|
|                use refrigerator|      DA|
|                         use fan|      DA|
|                        use oven|      DA|
|                       use stove|      DA|
|     i would like to wash sheets|      DA|
|i would like to watch television|      DA|
+--------------------------------+--------+
only showing top 10 rows



In [6]:
from pyspark.sql.functions import col
trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      VC|   46|
|      DA|   29|
|      CC|   26|
|      SD|   23|
|      LR|   21|
|      AR|   20|
|      SN|   20|
|     NAV|   18|
|      NA|    1|
+--------+-----+



In [7]:
testDataset = spark.read \
      .option("header", True) \
      .csv("text_test.csv")
testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      VC|   46|
|      DA|   29|
|      CC|   26|
|      SD|   23|
|      LR|   21|
|      AR|   20|
|      SN|   20|
|     NAV|   18|
|      NA|    1|
+--------+-----+



In [8]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(500)\
    .setLr(0.003)\
    .setBatchSize(32)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('classifer_logs')\

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

use_pipelineModel = use_clf_pipeline.fit(trainDataset)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ / ]Download done! Loading the resource.
[ \ ]

2023-02-23 22:32:37.969037: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]


2023-02-23 22:32:45.288840: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/1f0e9aaf6060_classifier_dl10587189560485087579
2023-02-23 22:32:45.316880: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2023-02-23 22:32:45.316900: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/1f0e9aaf6060_classifier_dl10587189560485087579
2023-02-23 22:32:45.566199: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-02-23 22:32:45.962215: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/1f0e9aaf6060_classifier_dl10587189560485087579
2023-02-23 22:32:46.063830: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 775000 microsec

Training started - epochs: 500 - learning_rate: 0.003 - batch_size: 32 - training_examples: 164 - classes: 9
Epoch 1/500 - 0.21s - loss: 13.56956 - acc: 0.2875 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.16s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           6	 4	 3	 0.6	 0.6666667	 0.63157904
NAV          0	 0	 3	 0.0	 0.0	 0.0
SD           0	 0	 5	 0.0	 0.0	 0.0
VC           4	 26	 0	 0.13333334	 1.0	 0.23529413
CC           0	 0	 7	 0.0	 0.0	 0.0
AR           0	 0	 6	 0.0	 0.0	 0.0
SN           0	 0	 3	 0.0	 0.0	 0.0
LR           0	 0	 3	 0.0	 0.0	 0.0
tp: 10 fp: 30 fn: 30 labels: 8
Macro-average	 prec: 0.09166667, rec: 0.20833334, f1: 0.12731482
Micro-average	 prec: 0.25, recall: 0.25, f1: 0.25
Epoch 2/500 - 0.02s - loss: 12.534501 - acc: 0.48125 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           8	 5	 1	 0.615

...

Epoch 434/500 - 0.01s - loss: 9.269306 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 0.6	 0.6	 0.6
VC           3	 0	 1	 1.0	 0.75	 0.85714287
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           4	 2	 2	 0.6666667	 0.6666667	 0.6666667
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 1	 1	 0.6666667	 0.6666667	 0.6666667
tp: 33 fp: 7 fn: 7 labels: 8
Macro-average	 prec: 0.8385416, rec: 0.79375, f1: 0.8155313
Micro-average	 prec: 0.825, recall: 0.825, f1: 0.825
Epoch 435/500 - 0.01s - loss: 9.26903 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 0.

Epoch 448/500 - 0.01s - loss: 9.265473 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 0.6	 0.6	 0.6
VC           3	 0	 1	 1.0	 0.75	 0.85714287
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           4	 2	 2	 0.6666667	 0.6666667	 0.6666667
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 1	 1	 0.6666667	 0.6666667	 0.6666667
tp: 33 fp: 7 fn: 7 labels: 8
Macro-average	 prec: 0.8385416, rec: 0.79375, f1: 0.8155313
Micro-average	 prec: 0.825, recall: 0.825, f1: 0.825
Epoch 449/500 - 0.01s - loss: 9.265206 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 0

Epoch 461/500 - 0.01s - loss: 9.2620325 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 0.6	 0.6	 0.6
VC           3	 0	 1	 1.0	 0.75	 0.85714287
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           4	 2	 2	 0.6666667	 0.6666667	 0.6666667
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 1	 1	 0.6666667	 0.6666667	 0.6666667
tp: 33 fp: 7 fn: 7 labels: 8
Macro-average	 prec: 0.8385416, rec: 0.79375, f1: 0.8155313
Micro-average	 prec: 0.825, recall: 0.825, f1: 0.825
Epoch 462/500 - 0.01s - loss: 9.261774 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 

Epoch 475/500 - 0.01s - loss: 9.258464 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 0.6	 0.6	 0.6
VC           3	 0	 1	 1.0	 0.75	 0.85714287
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           4	 2	 2	 0.6666667	 0.6666667	 0.6666667
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 1	 1	 0.6666667	 0.6666667	 0.6666667
tp: 33 fp: 7 fn: 7 labels: 8
Macro-average	 prec: 0.8385416, rec: 0.79375, f1: 0.8155313
Micro-average	 prec: 0.825, recall: 0.825, f1: 0.825
Epoch 476/500 - 0.01s - loss: 9.258212 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 0

Epoch 488/500 - 0.01s - loss: 9.25525 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 0.6	 0.6	 0.6
VC           3	 0	 1	 1.0	 0.75	 0.85714287
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           4	 2	 2	 0.6666667	 0.6666667	 0.6666667
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 1	 1	 0.6666667	 0.6666667	 0.6666667
tp: 33 fp: 7 fn: 7 labels: 8
Macro-average	 prec: 0.8385416, rec: 0.79375, f1: 0.8155313
Micro-average	 prec: 0.825, recall: 0.825, f1: 0.825
Epoch 489/500 - 0.01s - loss: 9.255005 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 2	 2	 0.

In [9]:
use_pipelineModel.stages[2].write().overwrite().save('Text_Classification')

In [10]:
#predictions =  use_pipelineModel.transform(testDataset)
#predictions.select("category", "text", "class.result").show(5, truncate=30)

In [11]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
loaded_ner_model = ClassifierDLModel.load("Text_Classification")\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        loaded_ner_model
    ])

#use_pipelineModel = use_clf_pipeline.fit(trainDataset)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [12]:
from sparknlp.base import LightPipeline
#clf_pipelineModel = use_clf_pipeline.fit(trainDataset)

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = use_clf_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['class'])

{'document': ['navigate to kitchen'], 'sentence_embeddings': ['navigate to kitchen'], 'class': ['NAV']}
['NAV']


In [13]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['class'])
    text = input("Enter New Text\n")

Enter Testing Text
quit
['SD']
Enter New Text
exit
